# Drowsiness Detection and Employee Alerting System

In [1]:
import jedi
%config Completer.use_jedi = True
from scipy.spatial import distance
from imutils import face_utils
import imutils
import dlib
import cv2
import numpy as np
import pandas as pd
from time import sleep
import easygui
import socket
import paho.mqtt.publish as publish
import winsound
import timeit

In [2]:
def eye_aspect_ratio(value):
    p2_p6 = distance.euclidean(value[1],value[5])
    p3_p5 = distance.euclidean(value[2],value[4])
    p1_p4 = distance.euclidean(value[0],value[3])
    EAR = (p2_p6 + p3_p5) / (2 * p1_p4)
    return EAR

In [3]:
threshold_eye = 0.25
threshold_mouth = 20
frames = 20
face_detect = dlib.get_frontal_face_detector()
face_predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [4]:
(left_index_1,left_index_2) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(right_index_1,right_index_2) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(mouth_index_1,mouth_index_2) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

In [5]:
HOST = "192.168.0.6"
PORT = 5454
frequency = 2500  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
capture_video = cv2.VideoCapture(0)
access = capture_video.isOpened()

if not access:
    print("Allow webcam to access this application")
    
my_list=[]
my_list_1=[]
my_list_2=[]

while access:
    ret, frame = capture_video.read()
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    eye_view = face_detect(gray,0)
    for i in eye_view:
        face_shape = face_predict(gray,i)
        face_shape = face_utils.shape_to_np(face_shape)
        
        leftEye = face_shape[left_index_1:left_index_2]
        rightEye = face_shape[right_index_1:right_index_2]
        
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0 
       
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        
        #s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        #s.connect((HOST,PORT))
        #if ear > 0:
            #s.sendto(ear,("192.168.0.6",5454))
        publish.single("EAR_Values", ear, hostname="test.mosquitto.org")
           
        if ear > threshold_eye:
            my_list_1.append(ear)
        else:
            my_list_2.append(ear)
            if len(my_list_2) == 150:
                winsound.Beep(frequency, duration)
                easygui.msgbox("Please wake up!!!!", title="Drowsiness Detection")
                my_list_2.clear()
                
               
    cv2.imshow('frame', frame) 
   
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        print("Camera Closed")
        break
        
capture_video.release()
cv2.destroyAllWindows()

Camera Closed
